# Lab: machine learning protocol, part 1

Author: Alasdair Newson

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error,accuracy_score
from sklearn.model_selection import cross_val_score,train_test_split

from sklearn.datasets import make_blobs

from sklearn.svm import SVC

# pour eviter les warnings embetants
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)



# 1/ Reproduceability and random seeds

We saw in the lesson that it was very important to be able to **reproduce** results. To get exactly the same results, in spite of using randomness in an algorithm, we look to **random seeds**. These are integer values which determine a sequence of **pseudo-random numbers** (see lesson).

To set this random seed in the numpy framework, we use:
- ```np.random.seed(rand_seed)```: https://numpy.org/doc/2.2/reference/random/generated/numpy.random.seed.html
  
If you put an empty input, then numpy will set the seed itself. It will use information from your computer to do this (that it hopes is random as possible). Use this function now, with a fixed seed, and check that a series of 5 random Gaussian variables is indeed the same each time. Note, to reset the random number generator back to the initial state, you have to set the seed again.

After this, use the default random seed (no input to ```np.random.seed```) and check to make sure that the series of random numbers is different.

In [ ]:
# STUDENT

## 2/ Train, test

We saw in the lesson that it is necessary to split up data into:
- train data, which we use for training the model (choosing the parameters)
- test data, which we use for testing (evaluating) the model. It is crucial that this data not be observed by the model during training

First let us create some random, 2D, data $X$ and binary labels $Y$. These data will contain two subsets of Gaussian data, one centred at:
- $\mu_1 = (0,0)$ with label 0
and one centred at:
- $\mu_2 = (2,2)$ with label 1

This can be done with the following function:

```make_blobs```: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html

Create this data now, with a total of $n=1000$ samples.

In [ ]:
# STUDENT

Now, use a scatter plot to display the data in $X$. Colour the data using the labels (blue for label 0 and red for label 1, for example).

In [ ]:
# STUDENT

Now we are ready to separate the data into train and test. There is a function to do this in scikit-learn:
```from sklearn.model_selection import train_test_split``` https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

Use the following (common) split:
- $80\%$ train
- $20\%$ test

Carry out the split, and check that the percentages are indeed respected. Display the train data with a scatter plot and verify visually that it indeed ressembles the original data.


In [ ]:
# STUDENT

## 2.1 Standard classification pipeline

Let's say we wish to classify the above data. There are two classes, so it is a binary classification problem. There is a general syntax for using machine learning models for prediction in scikit-learn. Imagine there is a machine learning algorithm called "toto". To import, train and predict using a toto-type model, we would use the following syntax.

```
from sklearn.TotoClass import Toto
my_toto = Toto(arguments)
my_toto.fit(X,Y)
my_toto.predict(X)
```

Let's try to do the classification with a "Support Vector Machine". This is a classifier which attempts to separate data by drawing a line (in fact a hyperplane) between the two classes. To do this, use the following class:
```
SVC(kernel="linear")
```
Carry out the classification and compare your prediction, visually (with two subplots, for example), with the ground truth, on the test set.

In [ ]:

# STUDENT

Are these visual results coherent with the intuitive idea of how the SVM works ?

Now, we need a more rigorous evaluation of the results. To do this, we will use the prediction accuracy. This is defined as:

- $\frac{\# \mathrm{good~predictions} }{\# \mathrm{total~predictions}}$

This can be calculated with:
```from sklearn.metrics import accuracy_score```

Do this evaluation now.

In [ ]:
# STUDENT

We saw in the lesson that, in order to estimate the performance of a classification algorithm, it is necessary to split the data up further into train, validation and test. Indeed, in most applications, we consider that we do not have access to the test data. Therefore, we use part of the train data as a simulation of "test" data. This is named validation data.

However, we need a robust evaluation of the performance (we might get unlucky/lucky with the train/validation split). For this, we use ``k-fold validation''. This consists in splitting the train data into several groups, and using one of those groups as validation, and then rotatiing the group used for validation. The model is re-trained each time.

This is obviously quite labour-intensive, but can be done with the following function
```cross_val_score(...)```: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html

Do this now and display the scores and their statistics.


In [ ]:

# STUDENT

In general, more data gives more robust estimates of performance. To test this hypothesis, re-produce the whole process (create data, train-test split, and k-fold validation), now with $n=10000$.

In [ ]:
# STUDENT

Does this confirm the hypothesis (more data gives more robust performance estimation) ?

What do you think of this result ? Do you think the performance estimation is reliable.

## 3/ Overfitting and regularisation

In this section, we are going to look at the problem of **overfitting**. As explained in the lesson, this happens when a machine learning model is so well-trained on the test dataset, that it cannot generalise well to the test dataset. It happens in particular when we allow the model to have a lot of parameters.

Let us illustrate this on a regresion problem. First, we create some data, which we will try to regress (predict). This data will represent a quadratic function, perturbed by some "noise" $\epsilon \sim \mathcal{N}(0,\sigma^2)$:
- $Y = a_0+a_1 X+ a_2 X^2 + \epsilon$

where $X\in \mathbb{R}$. Logically, we know that the solution should be a quadratic polynomial, but as we will see, the situation may be more complicated due to overfitting.

In [ ]:
# CODE IS GIVEN
x_min = -5.
x_max = 5.
sigma = 1.5
a_0 = 0
a_1 = 1
a_2 = 1
n=50


X = np.linspace(x_min,x_max,n)
Y = a_0 + a_1*X + a_2*np.power(X,2)
X = np.reshape(X,(n,1))
# add some noise to make the problem more difficult
np.random.seed(5) 
Y = Y+sigma*np.random.randn(n)
plt.scatter(X,Y)

Now, we are going to create a polynomial model using Ordinary Least Squares (sometimes this is referred to a polynomial regression). Let $k$ be the order of the polynomial. Although we know that $k$ should be equal to 2, we will pretend we do not know this.

In this context, the exponents of $X$ ($X**0, X**1, X**2, X**3$, etc) represent the **features** which we will use for the model, and the $a_0, a_1, a_2$, etc) are the parameters of the model. Notice that, in this sense, the model is linear: the prediction is just a linear combination of the input features. For example, using a polynomial of order 3:
$Y = a_0+a_1X+a_2X**2+a_3X**3$
In fact, the features could be anything ($\cos(X), \exp(X)$ etc), and the model would still be "linear" in this sense (we still have an OLS).

So now, let us create the polynomial features of $X$. This can be done with the following syntax:

```
poly = PolynomialFeatures(k,include_bias=True)
poly_features = poly.fit_transform(X)
```

Create these features now, using order $k=2$ (the theoretical value).

In [ ]:
# STUDENT

Now, we carry out the OLS (using the data in $X$ for the moment, no train/test). This is done by calling the the ```LinearRegression()``` function of scikit-learn. Use the syntax given above in the "toto" model, and replace toto with the linear regressor. Carry out the prediction and call this estimation ```Y_hat```.

In [ ]:
# STUDENT

Now, we wish to evaluate the model. Since the problem is one of regression, we use the mean squared error (MSE) as a metric. This can be accessed with:

- ```from sklearn.metrics import mean_squared_error```

Evaluate the MSE of your prediction with the polynomial model (with $k=2).

In [ ]:
# STUDENT

Now, do the same regression, except increase the order of the polynomial to $k=20$. What do you notice ?

In [ ]:
# STUDENT

So, it looks like we can just increase the model order to get better results ! But of course, we ignored one important aspect: train and test. To evaluate if our model is really better (that is to say, it generalises better), we need to fit on the train data, and predict/evaluate on the test data. Split the data up into train and test now, as above (same splits). In this section of the lab work, we do not worry about validation.


In [ ]:
# STUDENT

Now, re-write your code such that the model is fitted on the train data, and predicted/evaluated on the test data. If you wish, you can create a function to do this so you do not have to re-code it many times.

Once you have done this, evaluate the model using different values of $k$. What do you find is the best one ?

In [ ]:
# STUDENT

What is your conclusion now on the best model to use ?

## 3.1 Regularisation

This situation is quite simple, since we know that the polynomial should be quadratic. However, in many machine learning situations, we want to give the model the flexibility to use many parameters if it needs to, but at the same time ask it to not use them unless necessary. This idea is implemented using "regularisation", as seen in the lesson. In practice, this usually means minimising some function $R$ of the parameters $\theta$.

$$
\mathcal{L}(\theta) =  \sum_i \lVert Y^{(i)} - f_\theta(X^{(i)} \rVert^2_2 + R(\theta)
$$

Different functions $R$ are commonly used:
- $R(\theta) = \sum_i (\theta_i)^2$: "Ridge" regression (also called "Tikhonov" regularisation)
- $R(\theta) = \sum_i |\theta_i|$: "Lasso" regression (or "l1" regularisation)

These can be implemented, respectively, with the following functions:

- ```Ridge```: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html
- ```Lasso```: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html

Use these two now, setting $k=20$, and compare the regression results (standard, ridge and lasso). You can re-define a new function if you wish to take into account the different regularisations.

In [ ]:
# STUDENT

In [ ]:

# STUDENT

What are your conclusions ? Which regression works the best ? For each model, plot the estimation to see the effect of the regularisation (to do a plot instead of a scatter, you will need to order the elements in ```X_test```, otherwise the plot will be unreadable).

In [ ]:
# STUDENT

Try and change the regularisation parameter ($\alpha$) of the ridge and lasso regressions to see the effect of increasing the regularisation.

In [ ]:
# STUDENT

In [ ]:
# STUDENT